Just supply the HLS ID and this generates a DSWx-HLS product

In [1]:
from pystac_client import Client  
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
import requests
import concurrent.futures
from dem_stitcher import stitch_dem
from tile_stitcher import get_raster_from_tiles
import rasterio
import geopandas as gpd
import yaml
from shapely.geometry import box
from dotenv import dotenv_values
import datetime
import re
from dem_stitcher.rio_window import transform_bounds
from rasterio.crs import CRS
import json
import backoff

# Parameters

If you specify a directory for HLS layers, assume all the files are in a directory - directory should not mix products!

In [2]:
HLS_ID_OR_HLS_DIR_PATH = 'HLS.L30.T22UCU.2021266T142337.v2.0' # either all files are in a directory
OUT_DIR = 'out'
AEROSOL_PARAM = True
DOWNLOAD_HLS = True

# Accounting

In [3]:
if DOWNLOAD_HLS:
    HLS_ID = HLS_ID_OR_HLS_DIR_PATH
else:
    dir_path = Path(HLS_ID_OR_HLS_DIR_PATH)
    assert dir_path.exists()
    assert dir_path.is_dir()
    tifs = sorted(list((dir_path).glob('*.tif')))
    # HLS.L30.T22UCU.2021266T142337.v2.0.B01.tif
    HLS_ID = '.'.join(tifs[0].stem.split('.')[:-1])
HLS_ID

'HLS.L30.T22UCU.2021266T142337.v2.0'

In [4]:
OUT_DIR = Path(OUT_DIR)
OUT_DIR.mkdir(exist_ok=True, parents=True)

In [5]:
config = dotenv_values()

ED_USERNAME = config['ED_USERNAME']
ED_PASSWORD = config['ED_PASSWORD']

In [6]:
work_dir = Path('work') / HLS_ID
work_dir.mkdir(exist_ok=True, parents=True)

## DSWx ID

In [7]:
def get_dswx_id(hls_id: str):
    # Source: https://github.com/nasa/opera-sds-pcm/blob/83ed093f90fc7ed298f6bfc3a82f6aa48aece5e2/tools/ops/cnm_check.py#L84
    m = re.match(
        r'(?P<product_shortname>HLS[.]([LS])30)[.]'
        r'(?P<tile_id>T[^\W_]{5})[.]'
        r'(?P<acquisition_ts>(?P<year>\d{4})(?P<day_of_year>\d{3})T(?P<hour>\d{2})(?P<minute>\d{2})(?P<second>\d{2}))[.]'
        r'(?P<collection_version>v\d+[.]\d+)$',
        hls_id
    )
    tile = m.group("tile_id")
    year = m.group("year")
    doy = m.group("day_of_year")
    time_of_day = m.group("acquisition_ts").split("T")[1]
    date = datetime.datetime(int(year), 1, 1) + datetime.timedelta(int(doy) - 1)
    dswx_acquisition_dt_str = f"{date.strftime('%Y%m%d')}T{time_of_day}"

    now = datetime.datetime.now()
    now_str = now.strftime('%Y%m%d%H%S')
    dswx_id = f'OPERA_L3_DSWx-HLS_{tile}_{dswx_acquisition_dt_str}Z_{now_str}Z'

    return dswx_id

dswx_id = get_dswx_id(HLS_ID)
dswx_id

'OPERA_L3_DSWx-HLS_T22UCU_20210923T142337Z_202312210937Z'

In [8]:
product_version = 1.0

# This is where DSWx will get written
output_dir = OUT_DIR / f'{dswx_id}_{product_version}'
output_dir.mkdir(exist_ok=True, parents=True)

In [9]:
data = {'hls_id': HLS_ID,
        'dswx_product_dir': str(output_dir)}
json.dump(data,
          open(output_dir / 'hls_dswx_dict.json', 'w'))

# Download HLS Tile

In [10]:
hls_dir = work_dir / HLS_ID
hls_dir.mkdir(exist_ok=True, parents=True)

In [11]:
@backoff.on_exception(backoff.expo,
                      Exception,
                      max_tries=10)
def download_one(url: str, work_dir: Path = hls_dir):
    # Source: https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests
    local_filename = work_dir / url.split('/')[-1]
    session = requests.Session()
    session.auth = (ED_USERNAME, ED_PASSWORD)
    with session.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return local_filename

@backoff.on_exception(backoff.expo,
                      Exception,
                      max_tries=10)
def get_hls_urls(hls_id: str) -> list[str]:
    STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'
    api = Client.open(f'{STAC_URL}/LPCLOUD/')
    hls_collections = ['HLSL30.v2.0', 'HLSS30.v2.0']

    search_params = {"collections": hls_collections,
                     "ids": [HLS_ID],
                     "max_items": 5}
    resp = api.search(**search_params)
    resp_items = resp.item_collection()

    urls = [asset.href for asset in resp_items[0].assets.values() if asset.href[-4:] not in ['.xml', '.jpg']]
    return urls
    
def download_hls_data(hls_id: str) -> list[str]:
    urls = get_hls_urls(hls_id)
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        hls_paths = list(tqdm(executor.map(download_one, urls), total=len(urls)))
    return hls_paths

In [12]:
hls_paths = download_hls_data(HLS_ID)

100%|█████████████████████████████████| 15/15 [00:24<00:00,  1.63s/it]


# HLS Bounds (with a buffer)

Get bounds of HLS tile

In [13]:
with rasterio.open(hls_paths[0]) as ds:
    org_bounds = list(ds.bounds)
    hls_crs = ds.crs
org_bounds_4326 = transform_bounds(org_bounds, hls_crs, CRS.from_epsg(4326))

hls_bounds = box(*org_bounds_4326).buffer(.25).bounds
hls_bounds

(-53.917542329726025, 47.48415560187451, -51.97700553578177, 48.99648044425331)

# Auxiliary Datasets

## DEM

In [ ]:
dst_area_or_point = 'Point'
dem, p_dem = stitch_dem(hls_bounds, 
                        'glo_30', 
                        dst_area_or_point=dst_area_or_point, 
                        dst_ellipsoidal_height=True)

Reading glo_30 Datasets: 100%|██████████| 4/4 [00:07<00:00,  1.86s/it]


In [ ]:
dem_path = work_dir / 'glo_30.tif'

In [ ]:
with rasterio.open(dem_path, 'w', **p_dem) as ds:
   ds.write(dem, 1)
   ds.update_tags(AREA_OR_POINT=dst_area_or_point)

## ESA 10 m world cover

In [ ]:
X_esa_wc, p_wc = get_raster_from_tiles(hls_bounds, 
                                       tile_shortname='esa_world_cover_2021')

In [ ]:
wc_10m_path = work_dir / f'wc_10m.tif'

In [ ]:
with rasterio.open(wc_10m_path, 'w', **p_wc) as ds:
    ds.write(X_esa_wc)

# ESA 100 m World Cover

In [ ]:
wc_100m_path =  work_dir / f'wc_100m.tif'

In [ ]:
X_cop100, p_cop100 = get_raster_from_tiles(hls_bounds, 
                                           tile_shortname='cop_100_lulc_discrete',
                                           year=2019)

In [ ]:
with rasterio.open(wc_100m_path, 'w', **p_cop100) as ds:
    ds.write(X_cop100)

## NOAA GSHHS

Commented out because it's not necessary when ocean mapping is not applied.

In [ ]:
# %%time

# #coastline_url = 'http://www.soest.hawaii.edu/pwessel/gshhg/gshhg-shp-2.3.7.zip' # Source: https://www.soest.hawaii.edu/pwessel/gshhg/index.html
# coastline_url = ('https://asf-dem-west.s3.us-west-2.amazonaws.com'
#                  '/WATER_MASK/GSHHG/GSHHS_shp/f/GSHHS_f_L1.shp')

# df = gpd.read_file(coastline_url, bbox=hls_bounds)
# df.head()

In [ ]:
# coastline_path = work_dir / 'gshhs_coastline'
# coastline_path_shp = coastline_path / (coastline_path.stem + '.shp')

In [ ]:
# df.to_file(coastline_path)

## Delete all the data from memory

In [ ]:
del dem
# del df
del X_cop100
del X_esa_wc

# Setup Runconfig

In [ ]:
dswx_hls_runconfig_url = 'https://raw.githubusercontent.com/nasa/PROTEUS/main/src/proteus/defaults/dswx_hls.yaml'

In [ ]:
resp = requests.get(dswx_hls_runconfig_url)
runconfig_dict = yaml.safe_load(resp.content)
runconfig_dict

In [ ]:
hls_paths_str = [str(p.resolve()) for p in hls_paths]
hls_paths_str[:2]

In [ ]:
runconfig_dict['runconfig']['groups']['processing']['apply_aerosol_class_remapping'] = AEROSOL_PARAM

In [ ]:
runconfig_dict['runconfig']['groups']['input_file_group']['input_file_path'] = hls_paths_str
runconfig_dict['runconfig']['groups']['dynamic_ancillary_file_group']['dem_file'] = str(dem_path.resolve())
runconfig_dict['runconfig']['groups']['dynamic_ancillary_file_group']['landcover_file'] = str(wc_100m_path.resolve())
runconfig_dict['runconfig']['groups']['dynamic_ancillary_file_group']['worldcover_file'] = str(wc_10m_path.resolve())
#runconfig_dict['runconfig']['groups']['dynamic_ancillary_file_group']['shoreline_shapefile'] = str(coastline_path_shp.resolve())

In [ ]:
product_path = work_dir / 'product'
scratch_path = work_dir / 'scratch'


runconfig_dict['runconfig']['groups']['product_path_group']['product_path'] = str(product_path.resolve())
runconfig_dict['runconfig']['groups']['product_path_group']['scratch_path'] = str(scratch_path.resolve())
runconfig_dict['runconfig']['groups']['product_path_group']['product_version'] = product_version
runconfig_dict['runconfig']['groups']['product_path_group']['product_id'] = dswx_id
runconfig_dict['runconfig']['groups']['product_path_group']['output_dir'] = str(output_dir)

In [ ]:
runconfig_path = work_dir/ 'runconfig.yaml'
with open(runconfig_path, 'w') as f:
    yaml.dump(runconfig_dict, f, default_flow_style=False)

From the README - this is the recommended way - for debugging - it's easiest to write this to terminal. Ideally, we could use [this](https://github.com/nasa/PROTEUS/blob/main/tests/test_dswx_hls_workflow.py) to get python error-handling. But this worked and not messing.

**Note** this will require that the notebook is run from the environment `dswx_hls`!

In [ ]:
!dswx_hls.py {runconfig_path}